In [1]:
import sys
sys.path.append('../')

In [2]:
from scripts.utils import data_path
from scripts.cross_match_scripts import pandas_to_fits, desi_reliable_magnitudes
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
import seaborn as sns
%matplotlib inline

matplotlib settings set


matplotlib settings set


# Counterparts and field sources catalogs construction

## Load DESI-CSC 30 arcsec cone search match

In [3]:
desi_csc_orig = pd.read_pickle(data_path+'/csc_allsky_desi_r30_gaia_dered.gz_pkl',compression='gzip')

In [4]:
desi_csc_orig = desi_reliable_magnitudes(desi_csc_orig) #make cuts for SNR of flux + add colors
desi_csc_orig

/Users/sdbykov/opt/miniconda3/envs/venv_hea/lib/python3.10/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/sdbykov/opt/miniconda3/envs/venv_hea/lib/python3.10/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/sdbykov/opt/miniconda3/envs/venv_hea/lib/python3.10/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/sdbykov/opt/miniconda3/envs/venv_hea/lib/python3.10/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/sdbykov/opt/miniconda3/envs/venv_hea/lib/python3.10/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log10
  result = getattr

,csc_name,csc_ra,csc_dec,csc_flux_05_2,csc_radec_err,csc_r_98,release,objid,brickid,ra,dec,brick_primary,flux_g,flux_r,flux_z,flux_w1,flux_w2,flux_w3,flux_w4,flux_ivar_g,flux_ivar_r,flux_ivar_z,flux_ivar_w1,flux_ivar_w2,flux_ivar_w3,flux_ivar_w4,dered_mag_g,dered_mag_r,dered_mag_z,dered_mag_w1,dered_mag_w2,dered_mag_w3,dered_mag_w4,snr_g,snr_r,snr_z,snr_w1,snr_w2,snr_w3,snr_w4,parallax,parallax_ivar,pmra,pmra_ivar,pmdec,pmdec_ivar,dist_arcsec,desi_id,all_mag_g,rel_mag_g,rel_dered_mag_g,all_mag_r,rel_mag_r,rel_dered_mag_r,all_mag_z,rel_mag_z,rel_dered_mag_z,all_mag_w1,rel_mag_w1,rel_dered_mag_w1,vega_mag_w1,all_mag_w2,rel_mag_w2,rel_dered_mag_w2,vega_mag_w2,all_mag_w3,rel_mag_w3,rel_dered_mag_w3,vega_mag_w3,all_mag_w4,rel_mag_w4,rel_dered_mag_w4,vega_mag_w4,rel_desi_flux_corr_z,rel_dered_g_r,rel_dered_g_z,rel_dered_r_z,rel_dered_z_w1,rel_dered_r_w2,rel_dered_w1_w2,rel_dered_z_w3,rel_dered_r_w4,rel_dered_w3_w4
0,2CXO J000000.0+004331,0.000020,0.725285,2.625836e-15,0.492084,1.376434,9010,9135,336127,359.993680,0.721193,1,0.264035,0.761373,1.464151,-2.279967,-0.936121,-109.650260,-1274.12740,2924.99000,2268.87720,293.554100,3.076859,0.628511,0.000788,0.000010,23.857758,22.736668,22.052845,NaN,NaN,NaN,NaN,14.279862,36.266293,25.085913,-3.999284,-0.742145,-3.078044,-4.096002,0.0,0.0,0.0,0.0,0.0,0.0,27.164009,9010_336127_9135,23.945846,23.945846,23.857758,22.796006,22.796006,22.736668,22.086035,22.086035,22.052845,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.520281e-15,1.121090,1.804913,0.683823,NaN,NaN,NaN,NaN,NaN,NaN
1,2CXO J000000.0+004331,0.000020,0.725285,2.625836e-15,0.492084,1.376434,9010,9169,336127,359.994202,0.722597,1,0.994499,2.786103,5.748118,6.166792,6.704570,-0.933517,1110.99740,2478.61230,1837.17550,249.912500,2.884505,0.622227,0.000782,0.000011,22.417845,21.328133,20.567974,20.519806,20.430973,NaN,14.885468,49.511795,119.418724,90.869820,10.473576,5.288656,-0.026110,3.618951,0.0,0.0,0.0,0.0,0.0,0.0,23.070881,9010_336127_9169,22.505989,22.505989,22.417845,21.387507,21.387507,21.328133,20.601186,20.601186,20.567974,20.524852,20.524852,20.519806,17.825852,20.434073,20.434073,20.430973,17.095073,NaN,NaN,NaN,NaN,14.885717,NaN,NaN,NaN,9.894579e-15,1.089712,1.849871,0.760159,0.048168,0.897160,0.088833,NaN,NaN,NaN
2,2CXO J000000.0+004331,0.000020,0.725285,2.625836e-15,0.492084,1.376434,9010,9295,336127,359.997367,0.721166,1,0.287270,0.334849,0.528142,-1.608118,0.423848,16.828413,692.35394,2346.16870,1762.25840,213.479600,2.978809,0.614420,0.000778,0.000010,23.765974,23.628395,23.159851,NaN,23.428870,19.434230,15.398930,13.914554,14.056722,7.716652,-2.775487,0.332233,0.469412,2.225948,0.0,0.0,0.0,0.0,0.0,0.0,17.638166,9010_336127_9295,23.854274,23.854274,23.765974,23.687877,23.687877,23.628395,23.193123,23.193123,23.159851,NaN,NaN,NaN,NaN,23.431975,NaN,NaN,NaN,19.434892,NaN,NaN,NaN,15.399180,NaN,NaN,NaN,9.091730e-16,0.137579,0.606123,0.468544,NaN,NaN,NaN,NaN,NaN,NaN
3,2CXO J000000.0+004331,0.000020,0.725285,2.625836e-15,0.492084,1.376434,9010,9271,336127,359.996619,0.722544,1,0.195305,0.580013,1.060394,-1.374197,0.465793,55.299458,840.05010,1941.29630,1459.10840,181.554780,2.827259,0.595737,0.000749,0.000010,24.184935,23.031937,22.403067,NaN,23.326414,18.142536,15.188987,8.605151,22.155508,14.287992,-2.310637,0.359518,1.513426,2.704489,0.0,0.0,0.0,0.0,0.0,0.0,15.723989,9010_336127_9271,24.273217,24.273217,24.184935,23.091406,23.091406,23.031937,22.436332,22.436332,22.403067,NaN,NaN,NaN,NaN,23.329518,NaN,NaN,NaN,18.143198,NaN,NaN,NaN,15.189237,NaN,NaN,NaN,1.825409e-15,1.152998,1.781868,0.628870,NaN,NaN,NaN,NaN,NaN,NaN
4,2CXO J000000.0+004331,0.000020,0.725285,2.625836e-15,0.492084,1.376434,9010,9360,336127,359.998741,0.722629,1,0.162211,0.722331,4.141845,10.932156,10.346854,281.147600,-6404.08400,1451.76900,1055.47450,125.596010,2.283064,0.530093,0.000696,0.000010,24.386390,22.793606,20.923704,19.898174,19.959871,16.377000,NaN,6.180582,23.467144,46.417503,16.518280,7.533284,7.418694,-20.397125,0.0,0.0,0.0,

### Primary filters and DESI duplicates (ra,dec) removal

In [5]:
#step 1: filter brick ID
#next sort by name_csc, ra, dec and flux_g, with flux_g from largest to smallest
desi_csc = desi_csc_orig.query('brick_primary == True')
print('desi reliable catalog:', len(desi_csc))
desi_csc = desi_csc.sort_values(by=['csc_name', 'csc_ra', 'csc_dec', 'flux_g'], ascending=[True, True, True, False])

#step 2: remove duplicates in ['ra_csc', 'dec_csc', 'ra', 'dec'] and keep the one with the highest flux_g 
desi_csc = desi_csc.drop_duplicates(subset=['csc_ra', 'csc_dec', 'ra', 'dec'], keep='first')

print('desi reliable catalog after non-primary brick and duplicates removal:', len(desi_csc))


#drop LH area
ra_min = 154.763934
ra_max = 167.084550
dec_min = 54.124219
dec_max = 61.236185
in_lh_mask = desi_csc.eval('(csc_ra >= @ra_min) & (csc_ra <= @ra_max) & (csc_dec >= @dec_min) & (csc_dec <= @dec_max)')
desi_csc = desi_csc[~in_lh_mask]

#sort by sep_csc so that the closest objects are first
desi_csc.sort_values(by=['csc_name', 'dist_arcsec'], inplace=True)
desi_csc.sample(10)

desi reliable catalog: 3249397
desi reliable catalog after non-primary brick and duplicates removal: 3249392


,csc_name,csc_ra,csc_dec,csc_flux_05_2,csc_radec_err,csc_r_98,release,objid,brickid,ra,dec,brick_primary,flux_g,flux_r,flux_z,flux_w1,flux_w2,flux_w3,flux_w4,flux_ivar_g,flux_ivar_r,flux_ivar_z,flux_ivar_w1,flux_ivar_w2,flux_ivar_w3,flux_ivar_w4,dered_mag_g,dered_mag_r,dered_mag_z,dered_mag_w1,dered_mag_w2,dered_mag_w3,dered_mag_w4,snr_g,snr_r,snr_z,snr_w1,snr_w2,snr_w3,snr_w4,parallax,parallax_ivar,pmra,pmra_ivar,pmdec,pmdec_ivar,dist_arcsec,desi_id,all_mag_g,rel_mag_g,rel_dered_mag_g,all_mag_r,rel_mag_r,rel_dered_mag_r,all_mag_z,rel_mag_z,rel_dered_mag_z,all_mag_w1,rel_mag_w1,rel_dered_mag_w1,vega_mag_w1,all_mag_w2,rel_mag_w2,rel_dered_mag_w2,vega_mag_w2,all_mag_w3,rel_mag_w3,rel_dered_mag_w3,vega_mag_w3,all_mag_w4,rel_mag_w4,rel_dered_mag_w4,vega_mag_w4,rel_desi_flux_corr_z,rel_dered_g_r,rel_dered_g_z,rel_dered_r_z,rel_dered_z_w1,rel_dered_r_w2,rel_dered_w1_w2,rel_dered_z_w3,rel_dered_r_w4,rel_dered_w3_w4
492401,2CXO J020632.7-011805,31.636434,-1.301510,NaN,0.649389,1.816440,9010,5175,323294,31.635187,-1.295454,1,0.250307,0.293108,0.599035,-4.064476,-1.893079,55.555553,-11461.60900,1555.54090,1143.016100,143.012020,3.024029,0.664679,0.001248,0.000017,23.907005,23.767216,23.019892,NaN,NaN,18.137455,NaN,9.872212,9.909564,7.163718,-7.068017,-1.543386,1.962907,-47.532707,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,22.257146,9010_323294_5175,24.003818,24.003818,23.907005,23.832431,23.832431,23.767216,23.056370,23.056370,23.019892,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.138181,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.034261e-15,0.139789,0.887113,0.747324,NaN,NaN,NaN,NaN,NaN,NaN
1232724,2CXO J081815.6+205531,124.565348,20.925493,9.065188e-16,1.670082,4.671469,9010,4191,449277,124.567324,20.920159,1,2.674766,10.183252,35.299850,20.707910,10.151866,-25.458738,-333.97560,1470.03300,431.790280,126.218020,2.275122,0.574532,0.000766,0.000010,21.301197,19.892317,18.581364,19.202183,19.979044,NaN,NaN,102.553240,211.603550,396.582500,31.234793,7.694898,-0.704635,-1.064829,1.171611,2.189327,3.044459,0.887452,-7.382147,2.144470,20.319327,9010_449277_4191,21.431786,21.431786,21.301197,19.980284,19.980284,19.892317,18.630568,18.630568,18.581364,19.209659,19.209659,19.202183,16.510659,19.983635,19.983635,19.979044,16.644635,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.166537e-14,1.408880,2.719833,1.310953,-0.620819,-0.086727,-0.776861,NaN,NaN,NaN
2355873,2CXO J141937.5+532405,214.906530,53.401657,9.435726e-16,0.530975,1.485216,9011,1466,596965,214.906403,53.401714,1,0.066591,0.261323,1.454803,6.967803,3.961460,1.369710,207.55939,499.19788,186.644320,85.095690,5.658721,1.551314,0.002168,0.000026,25.409664,23.935640,22.081010,20.390440,21.004244,22.158190,16.707054,1.487831,3.570136,13.420168,16.575058,4.934068,0.063780,1.051881,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.342760,9011_596965_1466,25.441461,NaN,NaN,23.957056,NaN,NaN,22.092990,22.092990,22.081010,20.392260,20.392260,20.390440,17.693260,21.005362,21.005362,21.004244,17.666362,22.158428,NaN,NaN,NaN,16.707144,NaN,NaN,NaN,2.455743e-15,NaN,NaN,NaN,1.690570,NaN,-0.613804,NaN,NaN,NaN
2095814,2CXO J125500.7+084426,193.753327,8.740734,3.288756e-15,0.565226,1.581023,9010,2113,381384,193.746756,8.738454,1,0.336161,0.513978,0.877008,0.785748,2.115279,-39.063637,605.28296,766.59490,264.262820,34.557280,3.049055,0.671560,0.000763,0.000010,23.607273,23.171202,22.613720,22.757421,21.683897,NaN,15.544888,9.307438,8.355312,5.155529,1.372037,1.733445,-1.079065,1.917767,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,24.781830,9010_381384_2113,23.683632,23.683632,23.607273,23.222639,23.222639,23.171202,22.642491,22.642491,22.613720,22.761792,NaN,NaN,NaN,21.686581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.545104,NaN,NaN,NaN,1.503484e-15,0.436071,0.993553,0.557482,NaN,NaN,NaN,NaN,NaN,NaN
2251468,2CXO J134210.1+282403,205.542088,28.401021,1.099597e-15,0.429981,1.202720,9012,9355,488845,205.537895,28.394746,1,30.249842,41.783130,38.492330,0.000000,0.000000,0.000000,0.00000,86.36662,67.962240,68.113434,0.000000,0.000

## Calculating source densities and false association radius (r_false) for each CSC source 

In [6]:
def annuli_area_deg2(r_in_arcsec, r_out_arcsec):
    r_in_deg = r_in_arcsec/3600
    r_out_deg = r_out_arcsec/3600
    area = np.pi*(r_out_deg**2 - r_in_deg**2)
    return area

def r_false(desi_rho_deg2, thresh = 0.03):
    ''' Belvedersky+ 2022 '''
    desi_rho_arcsec2 = desi_rho_deg2/(3600**2)
    return np.sqrt(-np.log(1-thresh)/(np.pi*desi_rho_arcsec2))


#make a dataframe of CSC sources
csc = pd.DataFrame(desi_csc.groupby(by = 'csc_name', ).agg({'csc_ra': np.mean, 'csc_dec': np.mean}))


#calculate total number of DESI sources withing 10-30 arcsec of each CSC source
num_src_desi_10_30 =  desi_csc.groupby(by = ['csc_name', pd.cut(desi_csc.dist_arcsec, [10, 30]) ]).size().unstack()


#calculate source densities for 10-30'' annulus
src_dens_deg2_10_30 = num_src_desi_10_30/annuli_area_deg2(10, 30)

#drop sources with  zeros in the 10-30'' source density (very few sources)
csc.drop(csc[src_dens_deg2_10_30 == 0].index, inplace=True)

#assign r_false for 10-30'' annulus density
csc['r_false_003_dens_10_30'] = r_false(src_dens_deg2_10_30)

csc.reset_index(inplace=True)

#assign r_false for each CSC source from csc dataframe to desi_csc dataframe
desi_csc = desi_csc.merge(csc[['csc_name','r_false_003_dens_10_30']], on='csc_name')

desi_csc.sample(10)

,csc_name,csc_ra,csc_dec,csc_flux_05_2,csc_radec_err,csc_r_98,release,objid,brickid,ra,dec,brick_primary,flux_g,flux_r,flux_z,flux_w1,flux_w2,flux_w3,flux_w4,flux_ivar_g,flux_ivar_r,flux_ivar_z,flux_ivar_w1,flux_ivar_w2,flux_ivar_w3,flux_ivar_w4,dered_mag_g,dered_mag_r,dered_mag_z,dered_mag_w1,dered_mag_w2,dered_mag_w3,dered_mag_w4,snr_g,snr_r,snr_z,snr_w1,snr_w2,snr_w3,snr_w4,parallax,parallax_ivar,pmra,pmra_ivar,pmdec,pmdec_ivar,dist_arcsec,desi_id,all_mag_g,rel_mag_g,rel_dered_mag_g,all_mag_r,rel_mag_r,rel_dered_mag_r,all_mag_z,rel_mag_z,rel_dered_mag_z,all_mag_w1,rel_mag_w1,rel_dered_mag_w1,vega_mag_w1,all_mag_w2,rel_mag_w2,rel_dered_mag_w2,vega_mag_w2,all_mag_w3,rel_mag_w3,rel_dered_mag_w3,vega_mag_w3,all_mag_w4,rel_mag_w4,rel_dered_mag_w4,vega_mag_w4,rel_desi_flux_corr_z,rel_dered_g_r,rel_dered_g_z,rel_dered_r_z,rel_dered_z_w1,rel_dered_r_w2,rel_dered_w1_w2,rel_dered_z_w3,rel_dered_r_w4,rel_dered_w3_w4,r_false_003_dens_10_30
2614327,2CXO J160835.7+655237,242.148837,65.877123,2.424734e-15,0.975502,2.728626,9011,3948,632778,242.131666,65.874119,1,0.213754,0.341519,0.583884,-1.555808,-3.045988,-13.326933,292.68472,496.36250,174.522370,39.814594,6.584707,2.121899,0.004210,0.000049,24.079205,23.601791,23.048010,NaN,NaN,NaN,16.333729,4.762271,4.511697,3.684237,-3.992310,-4.437011,-0.864749,2.056706,0.0,0.0,0.0,0.0,0.0,0.0,27.483214,9011_632778_3948,24.175214,24.175214,24.079205,23.666463,23.666463,23.601791,23.084184,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.334000,NaN,NaN,NaN,NaN,0.477414,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.163504
3111126,2CXO J232011.7+081141,350.048880,8.194789,1.294465e-15,0.650315,1.819029,9010,2748,379152,350.042469,8.195643,1,0.320897,0.010585,0.689403,-3.877874,-4.276916,-106.613430,-1351.72920,631.57030,120.163200,35.804880,2.831146,0.626551,0.000685,0.000009,23.466806,27.258234,22.803110,NaN,NaN,NaN,NaN,8.064488,0.116031,4.125195,-6.524915,-3.385391,-2.790067,-3.999305,0.0,0.0,0.0,0.0,0.0,0.0,23.047438,9010_379152_2748,23.734086,23.734086,23.466806,27.438273,NaN,NaN,22.903817,22.903817,22.803110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.262824e-15,NaN,0.663696,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.052429
2368591,2CXO J142727.1+343555,216.862967,34.598836,1.745066e-15,1.574496,4.404102,9011,3313,518392,216.864823,34.603224,1,0.540632,0.852696,0.756479,1.689466,-0.250652,-10.106766,431.35410,386.35864,173.580400,88.595950,5.093441,1.223676,0.002395,0.000028,23.121021,22.641540,22.785402,21.927952,NaN,NaN,15.912783,10.626658,11.234261,7.120393,3.812898,-0.277270,-0.494563,2.302388,0.0,0.0,0.0,0.0,0.0,0.0,16.726616,9011_518392_3313,23.167746,23.167746,23.121021,22.673014,22.673014,22.641540,22.803008,22.803008,22.785402,21.930626,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.912915,NaN,NaN,NaN,1.283589e-15,0.479481,0.335619,-0.143862,NaN,NaN,NaN,NaN,NaN,NaN,1.488360
1780707,2CXO J120052.7+725556,180.219878,72.932298,4.507061e-15,1.181205,3.304008,9011,924,647485,180.201573,72.928972,1,0.118431,0.276229,1.949185,10.103189,8.105683,21.276830,-137.08687,827.58044,214.973160,89.012024,5.478980,1.610098,0.002974,0.000037,24.744131,23.848190,21.748161,19.984720,20.225487,19.179691,NaN,3.406990,4.050056,18.389816,23.648756,10.285274,1.160266,-0.830358,0.0,0.0,0.0,0.0,0.0,0.0,22.749653,9011_647485_924,24.816337,NaN,NaN,23.896827,23.896827,23.848190,21.775367,21.775367,21.748161,19.988854,19.988854,19.984720,17.289854,20.228026,20.228026,20.225487,16.889026,19.180233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.336741e-15,NaN,NaN,2.100029,1.763441,3.622703,-0.240767,NaN,NaN,NaN,1.103797
2396709,2CXO J143147.3+343647,217.947334,34.613254,3.507795e-15,0.431600,1.207250,9011,1914,518396,217.944462,34.606835,1,0.538369,2.136420,5.048959,10.209319,7.922874,-21.245316,-2620.83940,191.61688,19.582739,52.844437,3.999345,1.056080,0.002305,0.000028,23.135883,21.651253,20.728273,19.975424,20.251513,NaN,NaN,7.452414,9.454171,36.702995,20.416967,8.142000,-1.020101,-13.954917,0.0,0.0,0.0,0.0,0.0,0.0,24.628374,

In [7]:
#calculate for each CSC source the number of DESI sources within r_false
tmp_df = pd.DataFrame(desi_csc.query('dist_arcsec <= r_false_003_dens_10_30').groupby(by = 'csc_name').dist_arcsec.apply('count'))
print(tmp_df.dist_arcsec.value_counts())
tmp_df.columns = ['num_srcs_sep_less_r_false']

#merge with desi_csc dataframe
desi_csc = desi_csc.merge(tmp_df, on='csc_name')

1    72179
2     1853
3       41
4        1
Name: dist_arcsec, dtype: int64


## Counterpart  assignment

In [8]:
desi_csc_ctps = desi_csc.query('dist_arcsec <= r_false_003_dens_10_30 & num_srcs_sep_less_r_false==1 & dist_arcsec < csc_r_98 ')

In [9]:
print('delete ', desi_csc_ctps.duplicated(subset='desi_id').sum(), ' duplicated desi_id')
desi_csc_ctps = desi_csc_ctps.drop_duplicates(subset='desi_id')
print('Final number of counterparts:', len(desi_csc_ctps))
desi_csc_ctps['is_counterpart'] = True
#dublicates will appear here if untreated
#desi_csc_ctps.pivot_table(values='name_csc', index='desi_id', aggfunc='count').sort_values(by = 'name_csc',ascending=False)

delete  1  duplicated desi_id
Final number of counterparts: 72021


## Field source assignment

In [10]:
desi_csc_field = desi_csc.query("dist_arcsec>=10 & dist_arcsec<=50 ")
print('number of possible field sources: ', len(desi_csc_field))

number of possible field sources:  1410131


In [11]:
tmp_df = desi_csc_field.groupby(by = 'desi_id', ).agg({'csc_name': 'count'})
clear_field_ids = tmp_df.index[tmp_df['csc_name']==1]
desi_csc_field = desi_csc_field[desi_csc_field.desi_id.isin(clear_field_ids)==True]
fraction_to_retain = 0.7
desi_csc_field = desi_csc_field.sample(frac=fraction_to_retain)
desi_csc_field['is_counterpart'] = False
desi_csc_field['csc_flux_05_2'] = np.nan
desi_csc_field['csc_r_98'] = np.nan

print(f'Final number of secure field sources  ({fraction_to_retain} of it): ', len(desi_csc_field))

Final number of secure field sources  (0.7 of it):  860394


In [15]:
len(desi_csc_ctps.query('csc_flux_05_2>3e-15'))

35806

In [16]:
len(desi_csc_ctps.query('csc_flux_05_2>3e-15 & csc_dec>32'))

12459

In [17]:
len(desi_csc_field.query('csc_dec>32'))

207917

## Saving the combined catalogs

In [18]:
desi_csc_training = pd.concat([desi_csc_ctps, desi_csc_field])
#save cat to pickle
desi_csc_training.to_pickle(data_path+'csc_allsky_desi_r30_gaia_dered_training.gz_pkl', compression='gzip')